In [3]:
import numpy as np
from scipy.io import loadmat
from random import choice


In [6]:
def prepare_data():
  data = loadmat("movies.mat")
  X = data["Y"]

  return X


def create_X_train(X: np.array, test_samples = 100):
  m, n = X.shape
  X_test = np.zeros((m, n))

  mm = list(range(m))
  nn = list(range(n))
  i = 0

  ab = []

  while i < test_samples:
    a = choice(mm)
    b = choice(nn)

    if X[a, b] != 0:
      X_test[a, b] = X[a, b]
      ab.append([a, b])

      i += 1

  X_train = X - X_test
  return X_train, ab


def create_random_init_UV(users_num, values_num, p):
  U_init = np.random.random(size=(users_num, p))
  V_init = np.random.random(size=(p, values_num))
  return U_init, V_init

def create_init_UV(users_num, values_num, p):

  U_init = np.ones((users_num, p))
  V_init = np.ones((p, values_num))
  return U_init, V_init



def error_fun(X: np.array, U: np.array, V: np.array):
  A = np.dot(U, V) - X
  a = A.flatten()
  return np.dot(a.T, a)



def gradient_U(X: np.array, U: np.array, V: np.array):
  grad = np.zeros(U.shape)
  m, p = U.shape

  for i in range(m):
    for k in range(p):

      temp_vect = (np.dot(U[i, :], V) - X[i, :])*V[k,:]
      grad[i, k] = temp_vect.sum()

  return grad


def gradient_V(X: np.array, U: np.array, V: np.array):
  grad = np.zeros(V.shape)
  p, n = V.shape

  for j in range(n):
    for k in range(p):

      temp_vect = (np.dot(U, V[:, j]) - X[:, j])*U[:,k]
      grad[k, j] = temp_vect.sum()

  return grad


def optimization(
    X: np.array,
    U_init: np.array,
    V_init: np.array,
    alpha0=0.1,
    eps=1e-6,
    max_steps=1000,
):
  U = U_init
  V = V_init
  grad_U = gradient_U(X, U_init, V_init)
  grad_V = gradient_V(X, U_init, V_init)
  norm_grad_U = np.linalg.norm(grad_U)
  norm_grad_V = np.linalg.norm(grad_V)

  steps_counter = 0

  while norm_grad_U > eps and norm_grad_V > eps:

    if steps_counter >= max_steps:
      break

    grad_U = gradient_U(X, U, V)
    U = U - alpha0 * grad_U

    grad_V = gradient_V(X, U, V)
    V = V - alpha0 * grad_V

    steps_counter += 1
    norm_grad_U = np.linalg.norm(grad_U)
    norm_grad_V = np.linalg.norm(grad_V)

  err = error_fun(X, U, V)

  print(
      f"Optimization task is over, number of iterations = {steps_counter}\nobjective function value = {err}\n||gradU|| = {norm_grad_U}\n||gradV|| = {norm_grad_V}\n\n"
  )

  with open("opt.txt", "w") as f:
      print(
          f"Optimization task is over, number of iterations = {steps_counter}\nobjective function value = {err}\n||gradU|| = {norm_grad_U}\n||gradV|| = {norm_grad_V}\n\n",
          file=f,
      )

  return U, V

In [7]:
X = prepare_data()
users_num, values_num = X.shape
print(f"number of users = {users_num}\nnumber of movies = {values_num}")

X_train, ab = create_X_train(X)

number of users = 1682
number of movies = 943


In [19]:
few_steps = 30
errors_few_steps_test = dict()
errors_few_steps_all = dict()
choosen_alphas = dict()

for p in range(1, 31):
  print(f"p = {p}")
  U_init, V_init = create_random_init_UV(users_num=users_num, values_num=values_num, p=p)
  error = error_fun(X_train, U_init, V_init)

  for i in range(100):
    U_init_, V_init_ = create_random_init_UV(users_num=users_num, values_num=values_num, p=p)
    error_ = error_fun(X_train, U_init, V_init)
    if error_ < error:
      U_init = U_init_
      V_init = V_init_
      error = error_

  print(f"initial error = {error}")

  alphas = [0.002, 0.001, 0.0005, 0.0002, 0.0001, 0.00005]

  for alpha in alphas:
    U, V = optimization(X, U_init, V_init, max_steps=2, alpha0=alpha)
    opt_error = error_fun(X_train, U, V)
    if opt_error < error:
      break

  print(f"alpha = {alpha}")
  choosen_alphas[p] = alpha

  U, V = optimization(X, U_init, V_init, max_steps=few_steps, alpha0=alpha)
  errors_few_steps_all[p] = error_fun(X, U, V)

  UV = np.dot(U, V)
  UV_ = UV.round(2)

  comparison = []
  for item in ab:
    a = item[0]
    b = item[1]
    comparison.append((item, X[a, b], UV_[a, b]))

    comp_err = 0
    for row in comparison:
      comp_err += (row[1] - row[2])**2

    errors_few_steps_test[p] = comp_err

  try:
    if errors_few_steps_test[p] > errors_few_steps_test[p-1] and errors_few_steps_all[p] > errors_few_steps_all[p-1] and choosen_alphas[p] == choosen_alphas[p-1]:
      break
  except:
    pass

p = 1
initial error = 1373573.0869821268
Optimization task is over, number of iterations = 2
objective function value = 968095.1951625623
||gradU|| = 6147.889911295061
||gradV|| = 2350.8688272310646


alpha = 0.002
Optimization task is over, number of iterations = 26
objective function value = 962292.5616368802
||gradU|| = 6.389466382049478e-07
||gradV|| = 3.6209073391238087e-06


p = 2
initial error = 1585427.438936211
Optimization task is over, number of iterations = 2
objective function value = 969494.9612036982
||gradU|| = 4818.047047389741
||gradV|| = 2860.5534239382164


alpha = 0.002
Optimization task is over, number of iterations = 30
objective function value = 902374.8675791748
||gradU|| = 28.10709809848717
||gradV|| = 25.42028602511304


p = 3
initial error = 1942060.9663897818
Optimization task is over, number of iterations = 2
objective function value = 994949.1497547934
||gradU|| = 8617.839733344177
||gradV|| = 3740.9901686407816


alpha = 0.002
Optimization task is over, 

In [20]:
print(f"errors_few_steps_test = \n{errors_few_steps_test}")
print(f"errors_few_steps_all = \n{errors_few_steps_all}")
print(f"choosen_alphas = \n{choosen_alphas}")

errors_few_steps_test = 
{1: 812.8125000000002, 2: 759.7456, 3: 701.5741000000002, 4: 692.1294000000001, 5: 655.6777999999999, 6: 659.0926, 7: 640.5875999999998, 8: 759.7881000000002, 9: 745.072, 10: 739.2710000000001, 11: 738.1588, 12: 736.6300000000001, 13: 717.0278000000003, 14: 731.6779000000002, 15: 726.4527, 16: 855.6733999999997, 17: 870.2684}
errors_few_steps_all = 
{1: 962292.5616368802, 2: 902374.8675791748, 3: 854899.9682236766, 4: 841905.4477420844, 5: 821079.4708242199, 6: 815093.5134400809, 7: 813007.8718312707, 8: 908674.3318636641, 9: 898204.8059976891, 10: 889362.629373167, 11: 896588.3975218739, 12: 887546.4977687591, 13: 894460.8020453453, 14: 890245.8735337368, 15: 877757.5430804656, 16: 1024918.4003884463, 17: 1025097.0846979222}
choosen_alphas = 
{1: 0.002, 2: 0.002, 3: 0.002, 4: 0.001, 5: 0.001, 6: 0.001, 7: 0.001, 8: 0.0005, 9: 0.0005, 10: 0.0005, 11: 0.0005, 12: 0.0005, 13: 0.0005, 14: 0.0005, 15: 0.0005, 16: 0.0002, 17: 0.0002}


In [21]:
p = 7

U_init, V_init = create_random_init_UV(users_num=users_num, values_num=values_num, p=p)
error = error_fun(X_train, U_init, V_init)

for i in range(100):
  U_init_, V_init_ = create_random_init_UV(users_num=users_num, values_num=values_num, p=p)
  error_ = error_fun(X_train, U_init, V_init)
  if error_ < error:
    U_init = U_init_
    V_init = V_init_
    error = error_

print(f"initial error = {error}")

alpha = choosen_alphas[p]
print(f"alpha = {alpha}")

U, V = optimization(X, U_init, V_init, max_steps=3000, alpha0=alpha)

UV = np.dot(U, V)
UV_ = UV.round(2)





initial error = 5569200.933679744
alpha = 0.001
Optimization task is over, number of iterations = 3000
objective function value = 767228.6117159193
||gradU|| = 0.004528120138419177
||gradV|| = 0.0021715337939638845




In [22]:
comparison = []
for item in ab:
  a = item[0]
  b = item[1]
  comparison.append((UV_[a, b], X[a, b], a, b))

  comp_err = 0
  for row in comparison:
    comp_err += (row[1] - row[2])**2

print(f"test error = {comp_err}")

test error = 22545295


In [23]:
with open("X_train.txt", "w") as f:
    for row in list(X_train):
        print(str(list(row)), file=f)

with open("X.txt", "w") as f:
    for row in list(X):
        print(str(list(row)), file=f)

with open("UV.txt", "w") as f:
    for row in list(UV_):
        print(str(list(row)), file=f)

In [26]:
comparison.sort(reverse=True)

for row in comparison:
  print(row)

(5.13, 5, 0, 329)
(4.4, 2, 312, 682)
(4.27, 3, 257, 935)
(4.24, 4, 94, 294)
(4.03, 4, 236, 183)
(4.02, 5, 131, 397)
(3.72, 4, 222, 302)
(3.72, 4, 196, 333)
(3.67, 3, 94, 647)
(3.67, 3, 76, 392)
(3.59, 5, 194, 157)
(3.52, 3, 299, 633)
(3.46, 5, 301, 531)
(3.44, 4, 160, 41)
(3.39, 3, 482, 390)
(3.36, 5, 124, 346)
(3.29, 4, 171, 637)
(3.06, 5, 317, 710)
(2.98, 3, 299, 89)
(2.95, 5, 96, 86)
(2.85, 4, 110, 795)
(2.66, 5, 356, 209)
(2.61, 4, 122, 290)
(2.59, 4, 256, 921)
(2.57, 3, 275, 213)
(2.54, 4, 315, 531)
(2.51, 4, 484, 377)
(2.49, 5, 500, 12)
(2.36, 4, 791, 183)
(2.32, 5, 630, 534)
(2.3, 5, 504, 746)
(2.24, 2, 762, 398)
(2.13, 5, 171, 396)
(2.13, 3, 244, 350)
(2.06, 3, 327, 190)
(1.99, 4, 422, 620)
(1.98, 2, 87, 451)
(1.96, 3, 403, 279)
(1.93, 4, 211, 183)
(1.82, 3, 548, 434)
(1.72, 2, 418, 842)
(1.69, 2, 615, 233)
(1.65, 4, 133, 464)
(1.61, 4, 59, 12)
(1.59, 3, 596, 74)
(1.56, 3, 53, 891)
(1.51, 5, 41, 335)
(1.48, 5, 662, 532)
(1.45, 5, 9, 710)
(1.45, 3, 327, 901)
(1.44, 5, 125, 716)


Dokonano rozkładu macierzy X o rozmiarze 1682x943 na macierze U i V o rozmiarach odpowiednio 1682x7 i 7x943, minimalizując normę Frobeniusa ||UV - X|| bez regularyzacji. Optymalne rozwiązanie zostało znalezione. Mimo to zbudowany model nie działa dobrze. Być może udałoby się uzyskać poprawę jakościu modelu przy znacznym zwiększeniu wymiaru macierzy U i V. Niestety sprawdzenie tego jest bardzo kosztowne obliczenowo przy stosowaniu metody gradientu prostego do rozwiązania zadania optymalizacji.

